# Interpolation of audiometric AC direct thresholds to obtain 1/3 octave thresholds

6-6-2021 GC

In [1]:
from math import trunc
from math import log10
from pathlib import Path
import numpy as np
import pandas as pd

In [2]:
def read_AC_thresholds():
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Implant/Primary_data/AC_measured_thresholds.csv'
    p_in = Path(f_in)   
    df = pd.read_csv(p_in, index_col='Study_ID')
    d = {'AC_125':'f_125_Hz', 'AC_250':'f_250_Hz', 'AC_500':'f_500_Hz', 'AC_750':'f_750_Hz', 'AC_1000': 'f_1000_Hz', 'AC_1500': 'f_1500_Hz', 'AC_2000': 'f_2000_Hz', 'AC_3000': 'f_3000_Hz', 'AC_4000': 'f_4000_Hz', 'AC_8000': 'f_8000_Hz'}
    df.rename(d, axis=1, inplace=True)
    return df

In [3]:
def read_REDD():
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Headband/constants.xlsx'
    p_in = Path(f_in)   
    df = pd.read_excel(p_in, sheet_name='REDD', header = 0, usecols='E:W', skiprows=[1, 2, 3, 4, 5], nrows=1).T
    df.insert(0, 'band_name', band_names())
    df.set_index('band_name', drop=True, inplace=True)
    df.index.name = None
    df = df.T
    s = df.iloc[0]
    return s

In [4]:
def band_names():
    # return list with column names for results table
    frequencies = [125, 160, 200, 250, 315, 400, 500, 630, 800, 1000, 1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300, 8000]
    l = list()
    for f in frequencies:
        st = 'f_' + str(f) + '_Hz'
        l.append(st)
    return l

In [5]:
def interpolation(fx, f1, f2, th1, th2):
    # function to interpolate threshold, frequency is log10 transformed
    xp = [log10(f1), log10(f2)]
    x = log10(fx)
    fp = [th1, th2]
    interpol = np.interp(x, xp, fp, left=None, right=None, period=None)
    return interpol

In [6]:
def interpoleted_columns():
    # interpolation matrix, and 'for' loop to traverse single column (frequencies)
    act = read_AC_thresholds()
    itpd = {160:[125, 250, 0, 1], 200:[125, 250, 0, 1], 315:[250, 500, 1, 2], 400:[250, 500, 1, 2], 
            630:[500, 1000, 2, 3], 800:[500, 1000, 2, 3], 1250:[1000, 2000, 3, 4], 1600:[1000, 2000, 3, 4], 
            2500:[2000, 4000, 4, 5], 3150:[2000, 4000, 4, 5], 5000:[4000, 8000, 5, 6], 6300:[4000, 8000, 5, 6]}
    itpm = pd. DataFrame(itpd).T
    itpm.reset_index(inplace=True)
    itpm.rename({'index':'f'}, axis = 1, inplace = True)
    itpm = itpm.T
    d = dict()
    for i in range(len(itpm.T)):
        s = itpm[i]
        fx = s['f']
        f1 = s[0]
        f2 = s[1]
        y1 = act.iloc[:, s[2]]
        y2 = act.iloc[:, s[3]]
        dict_per_freq = interpolated(fx, f1, f2, y1, y2)
        d.update(dict_per_freq)
    df = pd.DataFrame(d, index=act.index, dtype='int')
    return df

In [7]:
def interpolated(fx, f1, f2, s1, s2):
    # apply interpolation to single threshold values and 
    #make a dictionary with result for all subjects for a single frequency
    act = read_AC_thresholds()
    l = list()
    for i in act.index:
        interpolated = interpolation(fx, f1, f2, s1[i], s2[i])
        interpolated = round(interpolated, 1)
        l.append(interpolated)
    name = 'f_' + str(fx) +'_Hz'
    dict_id_interp = {name:l}
    return(dict_id_interp)

In [8]:
def correction_REDD(df):
    # calculation of thresholds at eardrum (correction with REDD)
    #idx = read_AC_thresholds().index
    redd_df = read_REDD()
    corrected = df.add(read_REDD(), axis = 1)    # add Series with REDD values = thresholds FL   
    corrected = ((corrected*1000).astype(int).astype(float))/1000
    corrected = corrected.round(decimals=1)
    return corrected

In [9]:
def write_res_in_master(df):
    # write results to csv file in Master directory
    fout = '/media/guido/LACIE/Cingle_Guido/Master/Implant/Calculated_data/AC_thresholds_third_HL.csv'
    pout = Path(fout)
    df.to_csv(pout)

In [10]:
def finalize_save_data():
    # function to join measured and interpolated data,
    # to invoque correction_REDD
    # to invoque the 'write' function
    intpld = interpoleted_columns()
    measured = read_AC_thresholds()
    complete = measured.join(intpld)
    complete = complete[band_names()]
    corrected = correction_REDD(complete)
    write_res_in_master(complete)
    #return corrected
    return complete

In [11]:
if __name__ == "__main__":
    result = finalize_save_data()

/tmp/ipykernel_6333/4288987973.py:21: FutureWarning: Could not cast to int64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(d, index=act.index, dtype='int')


In [12]:
result

,f_125_Hz,f_160_Hz,f_200_Hz,f_250_Hz,f_315_Hz,f_400_Hz,f_500_Hz,f_630_Hz,f_800_Hz,f_1000_Hz,f_1250_Hz,f_1600_Hz,f_2000_Hz,f_2500_Hz,f_3150_Hz,f_4000_Hz,f_5000_Hz,f_6300_Hz,f_8000_Hz
Study_ID,,,,,,,,,,,,,,,,,,,
34,5,6.8,8.4,10,8.3,6.6,5,5.0,5.0,5,6.6,8.4,10,19.7,29.7,40,44.8,49.8,55
35,15,11.4,8.2,5,8.3,11.8,15,15.0,15.0,15,15.0,15.0,15,27.9,41.2,55,55.0,55.0,55
43,20,18.2,16.6,15,15.0,15.0,15,13.3,11.6,10,6.8,3.2,0,11.3,22.9,35,30.2,25.2,20
47,10,8.2,6.6,5,5.0,5.0,5,6.7,8.4,10,11.6,13.4,15,15.0,15.0,15,13.4,11.7,10
50,10,10.0,10.0,10,6.7,3.2,0,0.0,0.0,0,0.0,0.0,0,4.8,9.8,15,15.0,15.0,15
53,35,33.2,31.6,30,26.7,23.2,20,16.7,13.2,10,10.0,10.0,10,10.0,10.0,10,11.6,13.3,15
54,10,10.0,10.0,10,11.7,13.4,15,13.3,11.6,10,11.6,13.4,15,21.4,28.1,35,47.9,61.2,75
56,15,15.0,15.0,15,15.0,15.0,15,15.0,15.0,15,18.2,21.8,25,36.3,47.9,60,61.6,63.3,65
59,10,8.2,6.6,5,6.7,8.4,10,11.7,13.4,15,11.8,8.2,5,9.8,14.8,20,15.2,10.2,5


In [13]:
correction_REDD(result)

,f_125_Hz,f_160_Hz,f_200_Hz,f_250_Hz,f_315_Hz,f_400_Hz,f_500_Hz,f_630_Hz,f_800_Hz,f_1000_Hz,f_1250_Hz,f_1600_Hz,f_2000_Hz,f_2500_Hz,f_3150_Hz,f_4000_Hz,f_5000_Hz,f_6300_Hz,f_8000_Hz
Study_ID,,,,,,,,,,,,,,,,,,,
34,27.1,24.7,26.0,25.7,21.3,18.4,15.7,14.1,14.4,14.2,15.2,18.9,24.9,37.1,45.0,52.9,58.2,67.5,68.0
35,37.1,29.3,25.8,20.7,21.3,23.6,25.7,24.1,24.4,24.2,23.6,25.5,29.9,45.3,56.5,67.9,68.4,72.7,68.0
43,42.1,36.1,34.2,30.7,28.0,26.8,25.7,22.4,21.0,19.2,15.4,13.7,14.9,28.7,38.2,47.9,43.6,42.9,33.0
47,32.1,26.1,24.2,20.7,18.0,16.8,15.7,15.8,17.8,19.2,20.2,23.9,29.9,32.4,30.3,27.9,26.8,29.4,23.0
50,32.1,27.9,27.6,25.7,19.7,15.0,10.7,9.1,9.4,9.2,8.6,10.5,14.9,22.2,25.1,27.9,28.4,32.7,28.0
53,57.1,51.1,49.2,45.7,39.7,35.0,30.7,25.8,22.6,19.2,18.6,20.5,24.9,27.4,25.3,22.9,25.0,31.0,28.0
54,32.1,27.9,27.6,25.7,24.7,25.2,25.7,22.4,21.0,19.2,20.2,23.9,29.9,38.8,43.4,47.9,61.3,78.9,88.0
56,37.1,32.9,32.6,30.7,28.0,26.8,25.7,24.1,24.4,24.2,26.8,32.3,39.9,53.7,63.2,72.9,75.0,81.0,78.0
59,32.1,26.1,24.2,20.7,19.7,20.2,20.7,20.8,22.8,24.2,20.4,18.7,19.9,27.2,30.1,32.9,28.6,27.9,18.0


In [14]:
read_REDD()

f_125_Hz     22.1
f_160_Hz     17.9
f_200_Hz     17.6
f_250_Hz     15.7
f_315_Hz     13.0
f_400_Hz     11.8
f_500_Hz     10.7
f_630_Hz      9.1
f_800_Hz      9.4
f_1000_Hz     9.2
f_1250_Hz     8.6
f_1600_Hz    10.5
f_2000_Hz    14.9
f_2500_Hz    17.4
f_3150_Hz    15.3
f_4000_Hz    12.9
f_5000_Hz    13.4
f_6300_Hz    17.7
f_8000_Hz    13.0
Name: 0, dtype: float64